In [77]:
#Packages usadas
#pip install lxml
#pip install mysql-connector

In [21]:
alvo = 'https://www.ncbi.nlm.nih.gov/nuccore/NM_020056.5' # Gene HLA-DQA2

import requests
r = requests.get(alvo)
print(r.content)

b'<?xml version="1.0" encoding="utf-8"?>\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n<html xmlns="http://www.w3.org/1999/xhtml" lang="en" xml:lang="en">\n    <head xmlns:xi="http://www.w3.org/2001/XInclude"><meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n    <!-- meta -->\n    <meta name="robots" content="index,nofollow,noarchive" />\n<meta name="ncbi_app" content="entrez" /><meta name="ncbi_db" content="nuccore" /><meta name="ncbi_report" content="genbank" /><meta name="ncbi_format" content="html" /><meta name="ncbi_pagesize" content="20" /><meta name="ncbi_sortorder" content="default" /><meta name="ncbi_pageno" content="1" /><meta name="ncbi_resultcount" content="1" /><meta name="ncbi_op" content="retrieve" /><meta name="ncbi_pdid" content="genbank" /><meta name="ncbi_sessionid" content="CE8C00E05A2BB751_0447SID" /><meta name="ncbi_uidlist" content="1653961981" /><meta name="ncbi_fi

In [58]:
# Com este código criamos a função get_xml() para captura a informação da página
def get_xml(nuccore_id):

    import requests
    url = 'https://www.ncbi.nlm.nih.gov/nuccore/'+nuccore_id
    return requests.get(url)


Com o request acima não é possível obter informação útil, pelo que será necessário fazer o *parsing* do website.
Com recurso à função Inspect Element do browser e analisando os pacotes trocados no separador Network percebemos que o ficheiro GenBank é extraído através de uma solicitação pelo link:

`https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id=1653961981&conwithfeat=on&withparts=on&show-sequence=on&hide-cdd=on&ncbi_phid=CE8B8E325A26C7610000000006A205E5` de onde destacam-se os seguintes parâmetros:

`db` -> base de dados (neste caso nuccore)
`id` -> identificador do ficheiro genbank

Com recurso ao `BeautifulSoup` procuramos este ID nas meta tags da página de modo a poder sistematizar o scrapping de informação de ficheiros GenBank diretamente através do link e analisamos as meta tags em busca de uma que corresponda ao ID. 

Numa primeira análise encontramos o campo `'ncbi_uidlist'`, que isolamos finalmente e capturamos o ID recorrendo ao método `attrs` para conseguir ir buscar informação aos atributos da tag.

In [70]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content, 'lxml-xml')
soup.find_all('meta')

[<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>,
 <meta content="index,nofollow,noarchive" name="robots"/>,
 <meta content="entrez" name="ncbi_app"/>,
 <meta content="nuccore" name="ncbi_db"/>,
 <meta content="genbank" name="ncbi_report"/>,
 <meta content="html" name="ncbi_format"/>,
 <meta content="20" name="ncbi_pagesize"/>,
 <meta content="default" name="ncbi_sortorder"/>,
 <meta content="1" name="ncbi_pageno"/>,
 <meta content="1" name="ncbi_resultcount"/>,
 <meta content="retrieve" name="ncbi_op"/>,
 <meta content="genbank" name="ncbi_pdid"/>,
 <meta content="CE8DE5A75A2A9251_1584SID" name="ncbi_sessionid"/>,
 <meta content="1653961981" name="ncbi_uidlist"/>,
 <meta content="all" name="ncbi_filter"/>,
 <meta content="false" name="ncbi_stat"/>,
 <meta content="false" name="ncbi_hitstat"/>,
 <meta content="CE8DE5A75A278C41000000000630057B.m_31" name="ncbi_phid"/>,
 <meta content="origin-when-cross-origin" name="referrer"/>]

In [71]:
id = soup.find_all('meta',{'name':'ncbi_uidlist'})[0].attrs['content']
alvo = f'https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id={id}&conwithfeat=on&withparts=on&show-sequence=on&hide-cdd=on&ncbi_phid=CE8B8E325A26C7610000000006A205E5'

Podemos finalmente ler diretamente o ficheiro GenBank da base de dados.

In [72]:
r = requests.get(alvo)

In [73]:
gb_file = r.text
gb_file

'LOCUS       NM_020056               1458 bp    mRNA    linear   PRI 25-DEC-2022\nDEFINITION  Homo sapiens major histocompatibility complex, class II, DQ alpha 2\n            (HLA-DQA2), mRNA.\nACCESSION   NM_020056 XM_005249050 XM_005272806 XM_005274854 XM_005275110\n            XM_005275385 XM_005275545\nVERSION     NM_020056.5\nKEYWORDS    RefSeq; MANE Select.\nSOURCE      Homo sapiens (human)\n  ORGANISM  Homo sapiens\n            Eukaryota; Metazoa; Chordata; Craniata; Vertebrata; Euteleostomi;\n            Mammalia; Eutheria; Euarchontoglires; Primates; Haplorrhini;\n            Catarrhini; Hominidae; Homo.\nREFERENCE   1  (bases 1 to 1458)\n  AUTHORS   Luck K, Kim DK, Lambourne L, Spirohn K, Begg BE, Bian W, Brignall\n            R, Cafarelli T, Campos-Laborie FJ, Charloteaux B, Choi D, Cote AG,\n            Daley M, Deimling S, Desbuleux A, Dricot A, Gebbia M, Hardy MF,\n            Kishore N, Knapp JJ, Kovacs IA, Lemmens I, Mee MW, Mellor JC,\n            Pollis C, Pons C, Ric

In [61]:
# Declaramos funções para sistematizar este workflow

def get_ncbi_uid(nuccore_id):
    
    fonte = get_xml(nuccore_id)

    from bs4 import BeautifulSoup
    soup = BeautifulSoup(fonte.content, 'lxml-xml')
    id = soup.find_all('meta',{'name':'ncbi_uidlist'})[0].attrs['content']
    return id

def get_gb_file(nuccore_id):
    
    id = get_ncbi_uid(nuccore_id)
    src = f'https://www.ncbi.nlm.nih.gov/sviewer/viewer.cgi?tool=portal&save=file&log$=seqview&db=nuccore&report=genbank&id={id}&conwithfeat=on&withparts=on&show-sequence=on&hide-cdd=on&ncbi_phid=CE8B8E325A26C7610000000006A205E5'
    
    import requests
    r = requests.get(src)
    gb_file = r.text
    
    return gb_file


Opcionalmente podemos converter a informação em ficheiro para utilização posterior.

In [79]:
# Isolamos o accession number do gene com recurso a expressões regulares
# Sabemos que é a palavra a seguir a LOCUS, assumindo um ficheiro GenBank válido

import re

match = re.search(r'LOCUS\s+(.*?)\s+\b', gb_file)

if match:
    nuccore_id = match.group(1)
    print(f'O ID na base de dados Nucleotide é {nuccore_id}.')

# Criamos o ficheiro

filename = nuccore_id + '.gb'
with open(filename,'w',encoding='utf-8') as _:
    _.write(gb_file)
    print(f'Resultados gravados no ficheiro {filename}.')

O ID na base de dados Nucleotide é NM_020056.
Resultados gravados no ficheiro NM_020056.gb.


Definimos uma função para ler os resultados GenBank (seja ficheiro ou não) e capturar o ID do gene (LOCUS), a espécie de origem (SOURCE) e a sua sequência com recurso a expressões regulares.

In [70]:
def parse_genbank(nuccore_id):
    import re
    
    locus = get_gb_file(nuccore_id)
    
    i = re.match(r'LOCUS\s+(\w+)', locus)
    if i:
        id = i.group(1)
    organism = ""
    o = re.search(r'SOURCE\s+.+', locus)
    if o:
        s = re.match(r'SOURCE\s+(.+)', o[0] )
        if s:
            organism = s.group(1)
    sequencia = ""
    existe = re.findall(r'^\s+\d+ [actg ]+', locus, re.MULTILINE )
    if existe:
        for linha in existe:
            m = re.match( r'\s+\d+ (.+)', linha, re.DOTALL )
            # print( m.group(1) )
            # print( re.sub(r'\s+', '', m.group(1) ) ) 
            sequencia = sequencia + re.sub(r'\s+', '', m.group(1) )
    return (id, organism, sequencia)

# parse_genbank('NM_020056.gb')

# População de bases de dados com os resultados de múltiplas pesquisas

Conectamos à nossa base de dados usando o módulo mysql-connector e criamos a tabela `genbank` que iremos popular com os dados que iremos extrair dos ficheiros GenBank.

Concreteamente iremos utilizar como parâmetros:

* locusid como chave-primária, representando o id do gene
* dnasource, representanto a espécie a que este corresponde
* dnasequence, representando a sua sequência de nucléotidos

Estabelecemos conexão à base de dados:

In [1]:
# pip3 install mysql-connector
import mysql.connector as SQLC

DataBase = SQLC.connect(
   host ="127.0.0.1",
   user ="demo",
   password ="123456",
    database ="iapbd"
)

# Criação de um cursor para executar comandos SQL
Cursor = DataBase.cursor()

# Parâmetros para a criação da tabela
TableName ="""CREATE TABLE genbank
    (
    locusid varchar(255) PRIMARY KEY, 
    dnasource text, 
    dnasequence LONGTEXT
    );
"""

# Executamos os comandos com recurso ao cursor
Cursor.execute(TableName)


Confirmamos a existência da tabela vazia.

Recorremos ao módulos `pandas` para ter uma forma mais clara de visualizar a informação.

In [14]:
import pandas as pd

tabela = pd.read_sql("SELECT * FROM genbank",DataBase)

C:\Users\rsous\AppData\Local\Temp\ipykernel_19480\1356338761.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT * FROM genbank",DataBase)


In [15]:
tabela

,locusid,dnasource,dnasequence


In [16]:
InsertData = "INSERT INTO genbank (locusid, dnasource, dnasequence) VALUES (%s, %s, %s)"
val = parse_genbank('NM_020056.gb')

In [17]:
Cursor.execute(InsertData, val)

DataBase.commit()  # Executamos as modificações, fazendo commit.

print(Cursor.rowcount, "record inserted.")

1 record inserted.


Recorrendo ao módulo `pandas` podemos ver a tabela sobre a forma de um dataframe, sendo visualmente mais fácil de perceber a informação dentro de um Jupyter Notebook.

In [83]:
tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)
tabela

C:\Users\rsous\AppData\Local\Temp\ipykernel_19480\2230997112.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)


,locusid,comprimento
0,NM_020056,1458


Replicam-se estes passos abaixo para a inserção de vários genes (lista providenciada infra, com base no exemplo encontrado aqui).

In [ ]:
genes = [
    'L42022',
    'L42023',
    'LC740868.1'
]

# Verificamos todos os genes utilizando um ciclo for com recurso ao módulo time
# para não sobrecarregar o servidor

import time

vals = []
for gene in genes:

    vals.append(parse_genbank(gene))
    time.sleep(5)


In [85]:
# Com recurso a um ciclo adicionamos os valores à nossa base de dados


InsertData = "INSERT INTO genbank (locusid, dnasource, dnasequence) VALUES (%s, %s, %s)"

for val in vals:
    Cursor.execute(InsertData, val)
    
DataBase.commit()

Validação da inserção dos dados:

In [86]:
tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)
tabela

C:\Users\rsous\AppData\Local\Temp\ipykernel_19480\2230997112.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tabela = pd.read_sql("SELECT locusid, length(dnasequence) as comprimento FROM genbank",DataBase)


,locusid,comprimento
0,HIVI5C,231
1,L42023,1827071
2,LC740868,596
3,NM_020056,1458


In [87]:
# Fecho do cursos e conexão à base de dados

Cursor.close()
DataBase.close()